In [1]:
import pandas as pd
import gower
import numpy as np
from sklearn.cluster import AffinityPropagation
from sklearn import metrics

columns = ['estimatedp', 'wtr_srcdes', 'wtr_hauled', 'priv_wells', \
               'wc_exists', 'wc_adeq', 'wc_hlth', 'ww_public', 'ppl_nowat_r', 'ppl_noww_r', 'ppl_yeswat_r', 'ppl_yesww_r']

colonias_Y_path = 'dataset/colonias_Y_norm.csv'
colonias_N_path = 'dataset/colonias_N_norm.csv'

In [ ]:
df = pd.read_csv(colonias_Y_path)[columns]
selected_df = df[(df['ppl_nowat_r'] <= 1) & (df['ppl_noww_r'] <= 1)
                 & (df['ppl_yeswat_r'] <= 1) & (df['ppl_yesww_r'] <= 1)]

similarity_metric = gower.gower_matrix(selected_df)
preference = np.median(similarity_metric)
np.fill_diagonal(similarity_metric, preference)
similarity_metric *= -1

In [ ]:
dampings = [0.5, 0.6, 0.7, 0.8, 0.9]
iterations = range(200, 1050, 50)
for damping in dampings:
    for i in iterations:
        af = AffinityPropagation(affinity='precomputed', damping=damping, max_iter=i).fit(similarity_metric)
        n_clusters = len(np.unique(af.labels_))
        cluster_labels = af.labels_
        silhouette_avg = metrics.silhouette_score(similarity_metric, cluster_labels)
        print('Damping=', damping, ', iteration=', i, ' n_clusters=', n_clusters,
              'The average SScore=', silhouette_avg)